In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import datetime


In [ ]:
# Пункт 1
#Формируем входные данные
x  = tf.random.normal(shape=[100, 3])
y = tf.random.shuffle(tf.concat([tf.zeros(shape=[50, 1]),tf.ones(shape=[50, 1])], 0))

#Записываем их в входной массив
inputs = [x, y]

In [ ]:
# Пункт 2
class SecondModel(object):
  def __init__(self):
    self.W = tf.Variable(tf.random.normal(shape=[3, 1], stddev=0.3))
    self.b = tf.Variable(tf.random.normal(shape=[1]))

  def __call__(self, inputs):
    self.x = inputs[0]
    self.y = inputs[1]
    return self.x @ self.W + self.y + self.b

model = SecondModel()

In [ ]:
# Пункт 3
TRUE_W = tf.random.normal(shape=[3, 1], stddev=0.3)
TRUE_b = tf.random.normal(shape=[1])

noise = tf.random.normal(shape=[100, 1])
outputs = x @ TRUE_W + y + noise
print(f'TRUE_W: {TRUE_W}')
print(f'\nTRUE_b: {TRUE_b}')

In [ ]:
# Пунк 4
#Метод подсчета ошибки
def loss(target_y, predicted_y):
  return tf.reduce_mean(tf.abs(target_y - predicted_y))

In [ ]:
# Пункт 5
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

def train(model, inputs, outputs):
  with tf.GradientTape() as tape:
    current_loss = loss(outputs, model(inputs))
    grads = tape.gradient(current_loss, [model.W, model.b])
    optimizer.apply_gradients(zip(grads, [model.W, model.b]))

In [ ]:
# Пункт 6
! rm -R /tmp/mylogs/eager

In [ ]:
!ls '/tmp/mylogs/eager'

In [ ]:
PATH = '/tmp/mylogs/eager/'

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
loss_log_dir = PATH + current_time + '/data'
loss_summary_writer = tf.summary.create_file_writer(loss_log_dir)

model = SecondModel()

with loss_summary_writer.as_default():

  Ws = []
  loss_model = []
  epoch = 0
  speed_prev = 0
  #Добавляем обучение по эпохам
  while epoch < 10000:
    Ws.append(model.W.numpy().reshape(3))    
    current_loss = loss(outputs, model(inputs))    
    train(model, inputs, outputs)
    print('Epoch:',epoch,'. Weights:',Ws[-1],'. Loss:',current_loss.numpy())
    loss_model.append(current_loss)
    tf.summary.scalar("current_loss", current_loss, step=epoch)    
    tf.summary.scalar("W1", Ws[-1][0], step=epoch)    
    tf.summary.scalar("W2", Ws[-1][1], step=epoch)    
    tf.summary.scalar("W3", Ws[-1][2], step=epoch)    
    epoch+=1
    loss_summary_writer.flush()



plt.plot(range(epoch), loss_model, 'r')
plt.show()

In [ ]:
# Пункт 7
%load_ext tensorboard

In [ ]:
!ls /tmp/mylogs/eager

In [ ]:
%tensorboard --logdir '/tmp/mylogs/eager/'